1. Obtention des path de la BDD d'image SAR et Optique 

In [7]:
import gdal
import numpy as np
import os
import matplotlib.pyplot as plt
import rasterio

#import data_loader 

# path_bdd = "/work/OT/ai4geo/DATA/DATA_MULTIMODAL/"
path_bdd_France = "/work/OT/ai4geo/DATA/DATA_MULTIMODAL/France/"
path_bdd_USA = "/work/OT/ai4geo/DATA/DATA_MULTIMODAL/USA/"


def get_file_in_folder(folder):
    """
        Liste récursivement le contenu des sous-répertoires
    """
    list_file = []
    for f in os.listdir(folder):
        if os.path.isdir(folder+'/'+f): # si f est un dossier
            list_file.append(get_file_in_folder(folder+'/'+f))
        else :
            list_file.append(folder+'/'+f) 
    
    return(list_file)

# list_file = get_file_in_folder(path_bdd)
list_file_France = get_file_in_folder(path_bdd_France)
list_file_USA = get_file_in_folder(path_bdd_USA)

def select_file_name(list_file, word):
    list_selected = []
    for file in list_file:
        if type(file) is list:
            list_selected.append(select_file_name(file, word))
        elif str(file).find(word) != -1:
                list_selected.append(file)
    return list_selected

# list_SAR_file = select_file_name(list_file, 'S1moy')
# list_optique_file1 = select_file_name(list_file, 'S2')
# list_optique_file2 = select_file_name(list_file, 'S2mosa')
# list_optique_file = list_optique_file1 + list_optique_file2
# list_raster_file = select_file_name(list_file, 'OSM')
# list_SRTM_file = select_file_name(list_file, 'SRTM')
# list_S1_file = select_file_name(list_file, 'S1_')
list_SAR_file_France = select_file_name(list_file_France, 'S1moy')
list_SAR_file_USA = select_file_name(list_file_USA, 'S1moy')
list_optique_file_France = select_file_name(list_file_France, 'S2')
list_optique_file_USA = select_file_name(list_file_USA, 'S2mosa')

# for i in len list_
list_SAR_file_France[12].remove([])
list_optique_file_France[12].remove([])
list_SAR_file_France[21].remove([])
list_optique_file_France[21].remove([])
     

2. Création de la BDD

In [8]:
import os

from skimage import io
import numpy as np

import torch
from torch.utils.data import Dataset
import torchvision.transforms as T


#Define the data splits.
nb_train_data = 0.6
nb_val_data = 0.2
nb_test_data = 0.2

percent = 0.02

img_folder = [[] for i in range(3)]
SAR_folder = [[] for i in range(3)]
for i in range(len(list_SAR_file_France)):
    nb_imgs_zone = min(len(list_SAR_file_France[i]), len(list_optique_file_France[i]))
    sorted_SAR_list = sorted(list_SAR_file_France[i])
    sorted_optique_list = sorted(list_optique_file_France[i])
    for k in range(int(nb_imgs_zone*0.6*percent)):
        img_folder[0].append(sorted_SAR_list[k])  
        SAR_folder[0].append(sorted_optique_list[k]) 
    for k in range(int(nb_imgs_zone*0.6*percent),int(nb_imgs_zone*0.8*percent)):    
        img_folder[1].append(sorted_SAR_list[k])  
        SAR_folder[1].append(sorted_optique_list[k]) 
    for k in range(int(nb_imgs_zone*0.8*percent),int(nb_imgs_zone*percent)):
        img_folder[2].append(sorted_SAR_list[k])  
        SAR_folder[2].append(sorted_optique_list[k]) 

print(len(img_folder[0]))
print(len(SAR_folder[0]))   
print(img_folder[0][0])

class ImgOptiqueSAR(Dataset):
    def __init__(self, img_folder, SAR_folder, patch_size=256):
        self.imgs = []
        self.SARs = []

        #This will convert the numpy array to a tensor
        conversion = T.ToTensor()
        overlap = patch_size 

        for img_index in range(0,len(img_folder)):
            print("Working on image " + str(img_index))
            #Load the tile and the corresponding SAR truth.
            img = io.imread(img_folder[img_index])/2048
            SAR_open = gdal.Open(SAR_folder[img_index])
            SAR_band = SAR_open.GetRasterBand(1)
            SAR = SAR_band.ReadAsArray()

            for i in np.arange(patch_size//2, img.shape[0] - patch_size // 2 + 1, overlap):
                for j in np.arange(patch_size//2, img.shape[1] - patch_size // 2 + 1, overlap):
                      #Crop the image and the ground truth into patch around (i,j) and save
                      #them in self.imgs and self.SARs arrays.
                      #For the image, note that we are taking the three channels (using ":")
                      #for the 3rd dimension, and we do the conversion to tensor.
                      self.imgs.append(conversion(img[i - patch_size//2:i + patch_size // 2, j - patch_size // 2:j + patch_size // 2,:]))
                      self.SARs.append(SAR[i - patch_size//2:i + patch_size // 2, j - patch_size // 2:j + patch_size // 2])
 
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        
        img = self.imgs[idx].float()
        SAR = self.SARs[idx].float()

        return img, torch.from_numpy(SAR)

14
14
/work/OT/ai4geo/DATA/DATA_MULTIMODAL/France//Zone11/Couple11_S1moy_2016-08-29__dual_02_02.tif


3. Chargement de la BDD

In [27]:
batch_size = 2 #The batch size should generally be as big as your machine can take it.

training_dataset = ImgOptiqueSAR(img_folder[0], SAR_folder[0])
validate_dataset = ImgOptiqueSAR(img_folder[1], SAR_folder[1])

train_loader = torch.utils.data.DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
validate_loader = torch.utils.data.DataLoader(dataset=validate_dataset, batch_size=batch_size, shuffle=False)

print(train_loader.dataset.imgs[5].shape) # probleme --> taille d'une image 2*256*256 et pas 3*256*256

Working on image 0
Working on image 1
Working on image 2
Working on image 3
Working on image 4
Working on image 5
Working on image 6
Working on image 7
Working on image 8
Working on image 9
Working on image 10
Working on image 11
Working on image 12
Working on image 13
Working on image 0
Working on image 1
Working on image 2
Working on image 3
Working on image 4
Working on image 5
torch.Size([2, 256, 256])


4. Architecture Unet

5. Architecture PatchGAN

NameError: name 'Unet' is not defined